In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib import path
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy
import numpy as np
import netCDF4 as nc
np.set_printoptions(threshold=100000)
from shapely.geometry import Polygon, Point, MultiPoint
from shapely.ops import cascaded_union, unary_union, transform
from datetime import datetime, timedelta
import math
from scipy.ndimage.interpolation import shift
import shapely.wkt
from shapely.validation import explain_validity,make_valid
import xarray as xr
import pygeos as pg
import time
import seaborn as sns
from my_functions import sat_vap_press, vap_press, hot_dry_windy, haines
from joblib import Parallel, delayed
import multiprocessing
from os.path import exists

from timezonefinder import TimezoneFinder
import pytz
import time


In [49]:
sit209=pd.read_csv('../Query2.txt')
sit209.columns = ['INC209R_IDENTIFIER','INCIDENT_NAME','REPORT_FROM_DATE','REPORT_TO_DATE',
              'RESTYP_IDENTIFIER', 'RESOURCE_QUANTITY','RESOURCE_PERSONNEL','CODE_NAME',
              'INC_IDENTIFIER','PCT_CONTAINED_COMPLETED'] 

#all fires
#fire_incidents = ['BOBCAT', 'DOLAN', 'HOLIDAY FARM','CREEK', 'LAKE', 'CAMERON PEAK', 'PINE GULCH', 'WILLIAMS FLATS', 'SHADY','PEDRO MOUNTAIN', 'WALKER', '204 COW']

#2020 fires
fire_incidents = ['AUGUST COMPLEX','BOBCAT', 'DOLAN', 'HOLIDAY FARM','CREEK', 'LAKE', 'CAMERON PEAK', 'PINE GULCH']
#fire_incidents=['BOBCAT', 'DOLAN', 'HOLIDAY FARM','CREEK', 'LAKE', 'CAMERON PEAK', 'PINE GULCH']
#fire_incidents = ['LAKE']

path_poly = '/data2/lthapa/ML_daily/fire_polygons/'
suffix_poly = 'Z_day_start.geojson'
start_time=12
for jj in range(len(fire_incidents)):
    fire_name = fire_incidents[jj].lower().replace(' ','_')
    print(path_poly+fire_name+'_VIIRS_daily_'+str(start_time)+suffix_poly)
    fire_daily = gpd.read_file(path_poly+fire_name+'_VIIRS_daily_'+str(start_time)+suffix_poly)
    fire_daily=fire_daily.drop(columns=['Current Overpass'])
    fire_daily = fire_daily.drop(np.where(fire_daily['geometry']==None)[0])
    fire_daily['fire area (ha)'] = fire_daily['geometry'].area/10000 #hectares. from m2
    fire_daily.set_geometry(col='geometry', inplace=True) #designate the geometry column
    fire_daily = fire_daily.rename(columns={'Current Day':'UTC Day', 'Local Day': str(start_time)+ 'Z Start Day'})
    
    fire_daily = fire_daily.iloc[np.array(fire_daily['UTC Day'].values,dtype='datetime64')<=np.datetime64('2020-10-31'),:]

    
    #merra
    #me = merra_timeseries(fire_daily, start_time)
    #print(me)
    #me.to_csv('./fire_features/'+fire_name+'_Daily_MERRA_'+str(start_time)+'Z_day_start.csv') #daily averages

    #hrrr
    #hrrr = hrrr_timeseries(fire_daily,start_time)
    #print(hrrr)
    #hrrr.to_csv('./fire_features/'+fire_name+'_Daily_HRRR_'+str(start_time)+'Z_day_start.csv') #daily averages
    
    #hrrr with average wind
    #hrrr = hrrr_timeseries_ws(fire_daily,start_time)
    #print(hrrr)
    #hrrr.to_csv('./fire_features/'+fire_name+'_Daily_HRRR_ws_'+str(start_time)+'Z_day_start.csv') #daily averages
    
    #rave
    #rave=rave_timeseries(fire_daily,start_time)
    #print(rave)
    #rave.to_csv('./fire_features/'+fire_name+'_Daily_RAVE_'+str(start_time)+'Z_day_start.csv') #daily averages

    #pws
    #pws = pws_timeseries(fire_daily,start_time)
    #print(pws)
    #pws.to_csv('./fire_features/'+fire_name+'_Daily_PWS_'+str(start_time)+'Z_day_start.csv') #daily averages
   
    #hrrr_igbp
    #igbp =hrrr_igbp_timeseries(fire_daily,start_time)
    #print(igbp)
    #igbp.to_csv('./fire_features/'+fire_name+'_Daily_IGBP_'+str(start_time)+'Z_day_start.csv') #daily averages

    #esi 
    #esi = esi_timeseries(fire_daily,start_time)
    #print(esi)
    #esi.to_csv('./fire_features/'+fire_name+'_Daily_ESI_'+str(start_time)+'Z_day_start.csv') #daily averages
    
    #sit 209 data
    #resources = resources_timeseries(fire_daily,start_time,sit209)
    #print(resources)
    #resources.to_csv('./fire_features/'+fire_name+'_Daily_Resources_'+str(start_time)+'Z_day_start.csv')
    
    #fuel loading (takes 20 mins to process AC on the 1km grid)
    #fuel_loading = fuel_loading_timeseries(fire_daily,start_time)
    #print(fuel_loading)
    #fuel_loading.to_csv('./fire_features/'+fire_name+'_Daily_FUEL_LOADING_'+str(start_time)+'Z_day_start.csv') #daily averages
    
    #fire area
    fire_area = fire_area_timeseries(fire_daily,start_time)
    print(fire_area)
    fire_area.to_csv('./fire_features/'+fire_name+'_Daily_FIRE_AREA_'+str(start_time)+'Z_day_start.csv') #daily averages


/data2/lthapa/ML_daily/fire_polygons/august_complex_VIIRS_daily_12Z_day_start.geojson
           day  fire area (ha)  fire area (m2)  fire area (acre)
0   2020-08-16      462.348326    4.623483e+06       1142.462713
1   2020-08-17      308.435684    3.084357e+06        762.144575
2   2020-08-18    24154.183301    2.415418e+08      59684.986938
3   2020-08-19    19857.760979    1.985776e+08      49068.527378
4   2020-08-20    13023.242432    1.302324e+08      32180.432049
5   2020-08-21     8646.957847    8.646958e+07      21366.632841
6   2020-08-22     5823.917637    5.823918e+07      14390.900481
7   2020-08-23     1912.976370    1.912976e+07       4726.964610
8   2020-08-24     5610.498963    5.610499e+07      13863.542937
9   2020-08-25     3226.761171    3.226761e+07       7973.326852
10  2020-08-26     3318.487209    3.318487e+07       8199.981894
11  2020-08-27     4218.567013    4.218567e+07      10424.079089
12  2020-08-28     4343.039983    4.343040e+07      10731.651799
13  

           day  fire area (ha)  fire area (m2)  fire area (acre)
0   2020-09-07    10792.578143    1.079258e+08      26668.460590
1   2020-09-08    29824.069150    2.982407e+08      73695.274869
2   2020-09-09    15993.245350    1.599325e+08      39519.309260
3   2020-09-10     3429.773902    3.429774e+07       8474.971313
4   2020-09-11     1558.734776    1.558735e+07       3851.633630
5   2020-09-12     1051.709916    1.051710e+07       2598.775202
6   2020-09-13      479.076534    4.790765e+06       1183.798115
7   2020-09-14      939.721041    9.397210e+06       2322.050692
8   2020-09-15      411.234019    4.112340e+06       1016.159261
9   2020-09-16      923.746460    9.237465e+06       2282.577502
10  2020-09-17      125.059424    1.250594e+06        309.021837
12  2020-09-28        2.110240    2.110240e+04          5.214402
13  2020-10-03       15.199213    1.519921e+05         37.557255
14  2020-10-05        0.784476    7.844757e+03          1.938440
15  2020-10-06        7.1

           day  fire area (ha)  fire area (m2)  fire area (acre)
0   2020-07-31      218.783237    2.187832e+06        540.613379
1   2020-08-01      212.047765    2.120478e+06        523.970027
2   2020-08-02      365.701262    3.657013e+06        903.647819
3   2020-08-03      514.201092    5.142011e+06       1270.590900
4   2020-08-04     1978.795862    1.978796e+07       4889.604575
5   2020-08-05     1160.362477    1.160362e+07       2867.255680
6   2020-08-06      526.315535    5.263155e+06       1300.525687
7   2020-08-07     2143.291383    2.143291e+07       5296.073007
8   2020-08-08     1605.035890    1.605036e+07       3966.043684
9   2020-08-09     1875.028140    1.875028e+07       4633.194535
10  2020-08-10     2886.250809    2.886251e+07       7131.925749
11  2020-08-11     4027.979592    4.027980e+07       9953.137572
12  2020-08-12     3433.092426    3.433092e+07       8483.171386
13  2020-08-13     1607.881547    1.607882e+07       3973.075303
14  2020-08-14     1684.0

## Dataset-Dependent Helper Functions

### MERRA

In [ ]:
def merra_timeseries(df,day_start_hour):
    df_merra = pd.DataFrame({'day': np.zeros(len(df)),'temp':np.zeros(len(df)), 'vpd':np.zeros(len(df)), 
                             'wind':np.zeros(len(df)),'hd0w0':np.zeros(len(df)), 'hd1w0':np.zeros(len(df)),
                             'hd2w0':np.zeros(len(df)),'hd3w0':np.zeros(len(df)), 'hd4w0':np.zeros(len(df)),
                             'hd5w0':np.zeros(len(df))}) 
    
    #do the intersection, in parallel
    merra_intersections = Parallel(n_jobs=8)(delayed(calculate_intersection)
                                 (fire_daily.iloc[ii:ii+1],'MERRA_GRID',0.5) 
                                 for ii in range(len(fire_daily)))
    fire_merra_intersection=gpd.GeoDataFrame(pd.concat(merra_intersections, ignore_index=True))
    fire_merra_intersection.set_geometry(col='geometry')
    
    
    #loop over all of the days we have intersections
    times_intersect = np.unique(fire_merra_intersection[str(day_start_hour)+ 'Z Start Day'].values)
    times_utc = np.unique(fire_merra_intersection['UTC Day'].values)
    
    count = 0
    for today in times_intersect:
        #get the time
        df_sub = fire_merra_intersection.iloc[np.where(fire_merra_intersection[str(day_start_hour)+ 'Z Start Day'].values==today)]
        df_sub = df_sub.set_index([str(day_start_hour)+ 'Z Start Day', 'lat', 'lon'])
        intersection_sub = df_sub.to_xarray() #polygon and weights for today
        print(intersection_sub['weights'].sum(dim=['lat','lon']))
        times_back = pd.date_range(start=np.datetime64(today)-np.timedelta64(5,'D'), end=np.datetime64(today)+np.timedelta64(1,'D'))
        files_back = make_merra_file_namelist(times_back)
        
        #load in all the merra files associated with this lookback window
        dat_merra = xr.open_mfdataset(files_back,concat_dim='time',combine='nested',compat='override', coords='all')
    
        #add the derived data (svp, vp, vpd)
        dat_merra=dat_merra.assign(ESAT=sat_vap_press(dat_merra.TLML))
        dat_merra=dat_merra.assign(E=vap_press(dat_merra.QLML, dat_merra.TLML))
        dat_merra=dat_merra.assign(VPD=dat_merra.ESAT-dat_merra.E)
        
        merra_daily_mean = dat_merra.resample(time='24H',base=day_start_hour, label='left').mean(dim='time') #take the daily mean        
        merra_daily_mean_region = merra_daily_mean.sel(lat = np.unique(intersection_sub['lat'].values),
                                  lon = np.unique(intersection_sub['lon'].values)) #get the location of the overlaps
        
        hd0 = np.nansum((merra_daily_mean_region['VPD'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values)*(intersection_sub['weights'].values))
        hd1 = np.nansum((merra_daily_mean_region['VPD'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(1,'D')).values)*
                     (intersection_sub['weights'].values))
        hd2 = np.nansum((merra_daily_mean_region['VPD'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(2,'D')).values)*
                     (intersection_sub['weights'].values))
        hd3 = np.nansum((merra_daily_mean_region['VPD'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(3,'D')).values)*
                     (intersection_sub['weights'].values))
        hd4 = np.nansum((merra_daily_mean_region['VPD'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(4,'D')).values)*
                     (intersection_sub['weights'].values))
        hd5 = np.nansum((merra_daily_mean_region['VPD'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(5,'D')).values)*
                     (intersection_sub['weights'].values))
        w = np.nansum((merra_daily_mean_region['SPEEDLML'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values)*(intersection_sub['weights'].values))
        t = np.nansum((merra_daily_mean_region['TLML'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values)*(intersection_sub['weights'].values))
        
        df_merra.iloc[count,:] = [today+ ' '+str(day_start_hour)+':00:00',t,hd0,w,hd0*w,hd1*w,hd2*w,hd3*w,hd4*w,hd5*w]
        dat_merra.close()
        count =count+1
    return df_merra
    

In [ ]:
def make_merra_file_namelist(time):
    base_filename = '/data2/lthapa/YEAR/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.FULLDATE.nc4'
    base_filename_list = np.repeat(base_filename, len(time))

    
    for jj in range(len(time)):
        base_filename_list[jj] = base_filename_list[jj].replace('YEAR',time[jj].strftime('%Y')).\
                                    replace('FULLDATE',time[jj].strftime('%Y%m%d'))
        if (time[jj].strftime('%Y%m')=='202009'):
            base_filename_list[jj] = base_filename_list[jj].replace('400','401')
    return base_filename_list

### HRRR

In [ ]:
def hrrr_timeseries(df,day_start_hour):  
    df_hrrr_derived = pd.DataFrame({'day': np.zeros(len(df)),'hd0w0':np.zeros(len(df)), 'hd1w0':np.zeros(len(df)),
                             'hd2w0':np.zeros(len(df)),'hd3w0':np.zeros(len(df)), 'hd4w0':np.zeros(len(df)),
                             'hd5w0':np.zeros(len(df))})
    
    df_hrrr_raw = pd.DataFrame({'temp_2m':np.zeros(len(df)), 'q_2m':np.zeros(len(df)), 
                             'gust_sfc':np.zeros(len(df)),'veggie':np.zeros(len(df)), 'dewpt':np.zeros(len(df)),
                             'weasd':np.zeros(len(df)),'soilm':np.zeros(len(df)), 'esat_2m':np.zeros(len(df)),
                             'e_2m': np.zeros(len(df)),'vpd_2m':np.zeros(len(df)), 'hwp':np.zeros(len(df)), 
                             'gust_sfc':np.zeros(len(df)),'veggie':np.zeros(len(df)), 'dewpt':np.zeros(len(df)),
                             'veg_term':np.zeros(len(df)),'gust_max_term':np.zeros(len(df)), 'dd_term':np.zeros(len(df)),
                             'mois_term': np.zeros(len(df)),'temp_2m':np.zeros(len(df)), 'q_2m':np.zeros(len(df)), 
                             'gust_sfc':np.zeros(len(df)),'snowc_term':np.zeros(len(df))})
    
    varis = ['temp_2m', 'q_2m','gust_sfc','veggie', 'dewpt','weasd','soilm', 'esat_2m','e_2m',
             'vpd_2m', 'hwp','gust_sfc','veggie', 'dewpt','veg_term','gust_max_term', 'dd_term',
            'mois_term','temp_2m', 'q_2m','gust_sfc','snowc_term']
    
    #do the intersection, in parallel
    tic=time.time()
    hrrr_intersections = Parallel(n_jobs=8)(delayed(calculate_intersection)
                                 (fire_daily.iloc[ii:ii+1],'HRRR_GRID',0.05) 
                                 for ii in range(len(fire_daily)))
    fire_hrrr_intersection=gpd.GeoDataFrame(pd.concat(hrrr_intersections, ignore_index=True))
    fire_hrrr_intersection.set_geometry(col='geometry')
    toc = time.time()
    print(toc-tic)
    
    
    #loop over all of the days we have intersections
    times_intersect = np.unique(fire_hrrr_intersection[str(day_start_hour)+ 'Z Start Day'].values)
    times_utc = np.unique(fire_hrrr_intersection['UTC Day'].values)
    
    count = 0
    for today in times_intersect:
        print(today)
        #get the time
        df_sub = fire_hrrr_intersection.iloc[np.where(fire_hrrr_intersection[str(day_start_hour)+ 'Z Start Day'].values==today)]
        df_sub = df_sub.set_index([str(day_start_hour)+ 'Z Start Day', 'row', 'col'])
        df_sub=df_sub[~df_sub.index.duplicated()]
        intersection_sub = df_sub.to_xarray() #polygon and weights for today
        
        times_back = pd.date_range(start=np.datetime64(today)-np.timedelta64(5,'D'), end=np.datetime64(today)+
                                   np.timedelta64(1,'D'),freq='H')
        files_back,times_back_used = make_file_namelist(times_back,'/data2/lthapa/ML_daily/pygraf/Processed_HRRR_YYYYMMDDHH.nc')

        #load in all the merra files associated with this lookback window
        dat_hrrr = xr.open_mfdataset(files_back,concat_dim='time',combine='nested',compat='override', coords='all')
        dat_hrrr = dat_hrrr.assign_coords({'time': times_back_used})

        #add the derived data (svp, vp, vpd)
        dat_hrrr=dat_hrrr.assign(esat_2m=sat_vap_press(dat_hrrr.temp_2m))
        dat_hrrr=dat_hrrr.assign(e_2m=vap_press(dat_hrrr.q_2m, dat_hrrr.temp_2m))
        dat_hrrr=dat_hrrr.assign(VPD=dat_hrrr.esat_2m-dat_hrrr.e_2m)
        
        hrrr_daily_mean = dat_hrrr.resample(time='24H',base=day_start_hour, label='left').mean(dim='time') #take the daily mean        
        
        hrrr_daily_mean_region = hrrr_daily_mean.sel(grid_yt = np.unique(intersection_sub['row'].values),
                                                    grid_xt = np.unique(intersection_sub['col'].values)) #get the location of the overlaps
        print(hrrr_daily_mean_region['time'].values)
        hd0 = np.nansum((hrrr_daily_mean_region['vpd_2m'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values)*(intersection_sub['weights'].values))
        hd1 = np.nansum((hrrr_daily_mean_region['vpd_2m'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(1,'D')).values)*
                     (intersection_sub['weights'].values))
        hd2 = np.nansum((hrrr_daily_mean_region['vpd_2m'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(2,'D')).values)*
                     (intersection_sub['weights'].values))
        hd3 = np.nansum((hrrr_daily_mean_region['vpd_2m'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(3,'D')).values)*
                     (intersection_sub['weights'].values))
        hd4 = np.nansum((hrrr_daily_mean_region['vpd_2m'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(4,'D')).values)*
                     (intersection_sub['weights'].values))
        hd5 = np.nansum((hrrr_daily_mean_region['vpd_2m'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(5,'D')).values)*
                     (intersection_sub['weights'].values))
        w = np.nansum((hrrr_daily_mean_region['gust_sfc'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values)*(intersection_sub['weights'].values))
        t = np.nansum((hrrr_daily_mean_region['temp_2m'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values)*(intersection_sub['weights'].values))
        
        df_hrrr_derived.iloc[count,:] = [today+ ' '+str(day_start_hour)+':00:00',hd0*w,hd1*w,hd2*w,hd3*w,hd4*w,hd5*w]
        
        for var in varis:
            df_hrrr_raw[var].iloc[count] = np.nansum(intersection_sub['weights'].values*hrrr_daily_mean_region[var].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values, axis=(1,2))
        
        df_hrrr = pd.concat([df_hrrr_derived,df_hrrr_raw],axis=1)
        dat_hrrr.close()
        count =count+1
    return df_hrrr

In [14]:
def hrrr_timeseries_ws(df,day_start_hour):  #with the wind speed
    df_hrrr_derived = pd.DataFrame({'day': np.zeros(len(df)),'hd0w0':np.zeros(len(df)), 'hd1w0':np.zeros(len(df)),
                             'hd2w0':np.zeros(len(df)),'hd3w0':np.zeros(len(df)), 'hd4w0':np.zeros(len(df)),
                             'hd5w0':np.zeros(len(df))})
    
    df_hrrr_raw = pd.DataFrame({'temp_2m':np.zeros(len(df)), 'q_2m':np.zeros(len(df)), 
                             'wind_speed':np.zeros(len(df)),'veggie':np.zeros(len(df)), 'dewpt':np.zeros(len(df)),
                             'weasd':np.zeros(len(df)),'soilm':np.zeros(len(df)), 'esat_2m':np.zeros(len(df)),
                             'e_2m': np.zeros(len(df)),'vpd_2m':np.zeros(len(df)), 'hwp':np.zeros(len(df)), 
                             'veggie':np.zeros(len(df)), 'dewpt':np.zeros(len(df)),
                             'veg_term':np.zeros(len(df)),'gust_max_term':np.zeros(len(df)), 'dd_term':np.zeros(len(df)),
                             'mois_term': np.zeros(len(df)),'temp_2m':np.zeros(len(df)), 'q_2m':np.zeros(len(df)), 
                             'snowc_term':np.zeros(len(df))})
    
    varis = ['temp_2m', 'q_2m','wind_speed','veggie', 'dewpt','weasd','soilm', 'esat_2m','e_2m',
             'vpd_2m', 'hwp','veggie', 'dewpt','veg_term','gust_max_term', 'dd_term',
            'mois_term','temp_2m', 'q_2m','snowc_term']
    
    #do the intersection, in parallel
    tic=time.time()
    hrrr_intersections = Parallel(n_jobs=8)(delayed(calculate_intersection)
                                 (fire_daily.iloc[ii:ii+1],'HRRR_GRID',0.05) 
                                 for ii in range(len(fire_daily)))
    fire_hrrr_intersection=gpd.GeoDataFrame(pd.concat(hrrr_intersections, ignore_index=True))
    fire_hrrr_intersection.set_geometry(col='geometry')
    toc = time.time()
    print(toc-tic)
    
    
    #loop over all of the days we have intersections
    times_intersect = np.unique(fire_hrrr_intersection[str(day_start_hour)+ 'Z Start Day'].values)
    times_utc = np.unique(fire_hrrr_intersection['UTC Day'].values)
    
    count = 0
    for today in times_intersect:
        print(today)
        #get the time
        df_sub = fire_hrrr_intersection.iloc[np.where(fire_hrrr_intersection[str(day_start_hour)+ 'Z Start Day'].values==today)]
        df_sub = df_sub.set_index([str(day_start_hour)+ 'Z Start Day', 'row', 'col'])
        df_sub=df_sub[~df_sub.index.duplicated()]
        intersection_sub = df_sub.to_xarray() #polygon and weights for today
        
        times_back = pd.date_range(start=np.datetime64(today)-np.timedelta64(5,'D'), end=np.datetime64(today)+
                                   np.timedelta64(1,'D'),freq='H')
        files_back,times_back_used = make_file_namelist(times_back,'/data2/lthapa/ML_daily/pygraf/Processed_HRRR_YYYYMMDDHH.nc')

        #load in all the merra files associated with this lookback window
        dat_hrrr = xr.open_mfdataset(files_back,concat_dim='time',combine='nested',compat='override', coords='all')
        dat_hrrr = dat_hrrr.assign_coords({'time': times_back_used})

        #add the derived data (svp, vp, vpd)
        dat_hrrr=dat_hrrr.assign(esat_2m=sat_vap_press(dat_hrrr.temp_2m))
        dat_hrrr=dat_hrrr.assign(e_2m=vap_press(dat_hrrr.q_2m, dat_hrrr.temp_2m))
        dat_hrrr=dat_hrrr.assign(VPD=dat_hrrr.esat_2m-dat_hrrr.e_2m)
        
        hrrr_daily_mean = dat_hrrr.resample(time='24H',base=day_start_hour, label='left').mean(dim='time') #take the daily mean        
        
        hrrr_daily_mean_region = hrrr_daily_mean.sel(grid_yt = np.unique(intersection_sub['row'].values),
                                                    grid_xt = np.unique(intersection_sub['col'].values)) #get the location of the overlaps
        print(hrrr_daily_mean_region['time'].values)
        hd0 = np.nansum((hrrr_daily_mean_region['vpd_2m'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values)*(intersection_sub['weights'].values))
        hd1 = np.nansum((hrrr_daily_mean_region['vpd_2m'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(1,'D')).values)*
                     (intersection_sub['weights'].values))
        hd2 = np.nansum((hrrr_daily_mean_region['vpd_2m'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(2,'D')).values)*
                     (intersection_sub['weights'].values))
        hd3 = np.nansum((hrrr_daily_mean_region['vpd_2m'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(3,'D')).values)*
                     (intersection_sub['weights'].values))
        hd4 = np.nansum((hrrr_daily_mean_region['vpd_2m'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(4,'D')).values)*
                     (intersection_sub['weights'].values))
        hd5 = np.nansum((hrrr_daily_mean_region['vpd_2m'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(5,'D')).values)*
                     (intersection_sub['weights'].values))
        w = np.nansum((hrrr_daily_mean_region['wind_speed'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values)*(intersection_sub['weights'].values))
        t = np.nansum((hrrr_daily_mean_region['temp_2m'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values)*(intersection_sub['weights'].values))
        
        df_hrrr_derived.iloc[count,:] = [today+ ' '+str(day_start_hour)+':00:00',hd0*w,hd1*w,hd2*w,hd3*w,hd4*w,hd5*w]
        
        for var in varis:
            df_hrrr_raw[var].iloc[count] = np.nansum(intersection_sub['weights'].values*hrrr_daily_mean_region[var].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values, axis=(1,2))
        
        df_hrrr = pd.concat([df_hrrr_derived,df_hrrr_raw],axis=1)
        dat_hrrr.close()
        count =count+1
        print(df_hrrr)
    return df_hrrr

In [5]:
def make_hrrr_file_namelist(time):
    base_filename = '/data2/lthapa/ML_daily/pygraf/Processed_HRRR_YYYYMMDDHH.nc'
    base_filename_list = np.array([])
    times_back_used = np.array([])

    
    for jj in range(len(time)):
        fname = base_filename.replace('YYYYMMDDHH',time[jj].strftime('%Y%m%d%H'))
        if exists(fname):
            base_filename_list = np.append(base_filename_list,fname)
            times_back_used = np.append(times_back_used,time[jj])
            
    return base_filename_list, times_back_used

### RAVE

In [ ]:
def rave_timeseries(df, day_start_hour):
    varis = ['Mean_FRP', 'FRE', 'CO2', 'CO', 'SO2', 'OC', 'BC', 'PM2.5', 'NOx', 'NH3'] #don't need 'area', it's the area of each cell


    #do the intersection, in parallel
    rave_intersections = Parallel(n_jobs=8)(delayed(calculate_intersection)
                                 (fire_daily.iloc[ii:ii+1],'RAVE_GRID',0.1) 
                                 for ii in range(len(fire_daily)))
    fire_rave_intersection=gpd.GeoDataFrame(pd.concat(rave_intersections, ignore_index=True))
    fire_rave_intersection.set_geometry(col='geometry')    
    fire_rave_intersection = fire_rave_intersection.set_index([str(day_start_hour)+ 'Z Start Day', 'row', 'col'])
    
    fire_rave_intersection_xr = fire_rave_intersection.to_xarray()
    
    #load in rave data associated with the fire
    times = pd.date_range(np.datetime64(df[str(day_start_hour)+ 'Z Start Day'].iloc[0]),
                        np.datetime64(df[str(day_start_hour)+ 'Z Start Day'].iloc[len(df)-1])+
                        np.timedelta64(1,'D'))
    rave_filenames,times_back_used = make_file_namelist(times,'/data2/lthapa/YYYY/AprYYYY_to_OctYYYY/Hourly_Emissions_FV3_13km_YYYYMMDD0000_YYYYMMDD2300xr.nc')
    dat_rave = xr.open_mfdataset(rave_filenames,concat_dim='Time',combine='nested',compat='override', coords='all')
    dat_rave = dat_rave.assign_coords({'Time': dat_rave.time}) #assign coords so we can resample along time
    dat_rave = dat_rave.resample(Time='24H',base=day_start_hour).sum(dim='Time') #take the daily sum
    
    
    #select the locations and times we want
    dat_rave_sub = dat_rave.isel(yFRP = fire_rave_intersection_xr['row'].values.astype(int), 
                    xFRP = fire_rave_intersection_xr['col'].values.astype(int)).sel(
                    Time = pd.to_datetime(fire_rave_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values+
                                         'T'+str(day_start_hour)+':00:00')) #these should be lined up correctly
    ndays = len(fire_rave_intersection_xr[str(day_start_hour)+ 'Z Start Day'])
    
    #preallocate space for the output
    df_rave = pd.DataFrame({'day':np.zeros(ndays),'Mean_FRP':np.zeros(ndays),\
                          'FRE':np.zeros(ndays),  'CO2':np.zeros(ndays),  'CO':np.zeros(ndays),\
                          'SO2':np.zeros(ndays),  'OC':np.zeros(ndays),  'BC':np.zeros(ndays),\
                          'PM2.5':np.zeros(ndays),  'NOx':np.zeros(ndays),  'NH3':np.zeros(ndays)})

    df_rave['day'].iloc[:] = pd.to_datetime(fire_rave_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values+
                                         'T'+str(day_start_hour)+':00:00')
    for var in varis:
        df_rave[var] = np.nansum(fire_rave_intersection_xr['weights'].values*dat_rave_sub[var].values, axis=(1,2))
    
    return df_rave

In [ ]:
def make_rave_file_namelist(time):
    base_filename = '/data2/lthapa/YEAR/AprYEAR_to_OctYEAR/Hourly_Emissions_FV3_13km_FULLDATE0000_FULLDATE2300xr.nc'
    base_filename_list = np.repeat(base_filename, len(time))

    
    for jj in range(len(time)):
        base_filename_list[jj] = base_filename_list[jj].replace('YEAR',time[jj].strftime('%Y')).\
                                    replace('FULLDATE',time[jj].strftime('%Y%m%d'))
    return base_filename_list

### PWS

In [ ]:
def pws_timeseries(df, day_start_hour):
    #do the intersection, in parallel
    tic = time.time()
    pws_intersections = Parallel(n_jobs=8)(delayed(calculate_intersection)
                                 (fire_daily.iloc[ii:ii+1],'PWS_GRID',0.05) 
                                 for ii in range(len(fire_daily)))
    toc = time.time()
    print(toc-tic)
    fire_pws_intersection=gpd.GeoDataFrame(pd.concat(pws_intersections, ignore_index=True))
    fire_pws_intersection.set_geometry(col='geometry')
    fire_pws_intersection = fire_pws_intersection.set_index([str(day_start_hour)+ 'Z Start Day', 'lat', 'lon'])
    
    fire_pws_intersection_xr = fire_pws_intersection.to_xarray()
    
    #load in PWS data associated with the fire (it's only one dataset)
    
    #open the PWS files
    path_pws = '/data2/lthapa/PWS_6_jan_2021.nc'
    dat_pws = xr.open_dataset(path_pws) #map is fixed in time
    #print(dat_pws)
    
    dat_pws = dat_pws.assign_coords({'time': pd.to_datetime(fire_pws_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values)})
    dat_pws_daily = dat_pws['Band1'].expand_dims({'time': pd.to_datetime(fire_pws_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values)}) #the PWS expanded over all the days
    
    dat_pws_daily_sub = dat_pws_daily.sel(lat = fire_pws_intersection_xr['lat'].values, 
                                          lon = fire_pws_intersection_xr['lon'].values,
                      time = pd.to_datetime(fire_pws_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values), method='nearest')
                                          
    ndays = len(fire_pws_intersection_xr[str(day_start_hour)+ 'Z Start Day'])
    
    #preallocate space for the output
    df_pws = pd.DataFrame({'day':np.zeros(ndays),'PWS':np.zeros(ndays)})

    df_pws['day'].iloc[:] = pd.to_datetime(fire_pws_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values)
    varis=['PWS']
    for var in varis:
        df_pws[var] = np.nansum(fire_pws_intersection_xr['weights'].values*dat_pws_daily_sub.values, axis=(1,2))
    
    return df_pws
    

### HRRR IGBP Vegetation

In [ ]:
def hrrr_igbp_timeseries(df,day_start_hour):  
    df_igbp = pd.DataFrame({'day': np.zeros(len(df)),'Fraction_Evergreen_Needleleaf_Forests':np.zeros(len(df)),
                            'Fraction_Evergreen_Broadleaf_Forests':np.zeros(len(df)), 
                            'Fraction_Deciduous_Needleleaf_Forests':np.zeros(len(df)),
                            'Fraction_Deciduous_Broadleaf_Forests':np.zeros(len(df)), 
                            'Fraction_Mixed_Forests':np.zeros(len(df)),
                            'Fraction_Closed_Shrublands':np.zeros(len(df)),
                            'Fraction_Open_Shrublands':np.zeros(len(df)),
                            'Fraction_Woody_Savannas':np.zeros(len(df)),
                            'Fraction_Savannas':np.zeros(len(df)),
                            'Fraction_Grasslands':np.zeros(len(df)),
                            'Fraction_Permanent_Wetlands':np.zeros(len(df)),
                            'Fraction_Croplands':np.zeros(len(df)),
                            'Fraction_Urban_and_Builtup_Lands':np.zeros(len(df)),
                            'Fraction_Cropland_Natural_Vegetation_Mosaics':np.zeros(len(df)),
                            'Fraction_Permanent_Snow_and_Ice':np.zeros(len(df)),
                            'Fraction_Barren':np.zeros(len(df)),
                            'Fraction_Water_Bodies':np.zeros(len(df))})
    
    varis = ['Evergreen Needleleaf Forests','Evergreen Broadleaf Forests', 'Deciduous Needleleaf Forests',
         'Deciduous Broadleaf Forests','Mixed Forests','Closed Shrublands', 'Open Shrublands', 'Woody Savannas',
         'Savannas','Grasslands', 'Permanent Wetlands','Croplands','Urban and Builtup Lands', 
         'Cropland Natural Vegetation Mosaics','Permanent Snow and Ice','Barren','Water Bodies']

        
    #do the intersection, in parallel
    tic=time.time()
    hrrr_intersections = Parallel(n_jobs=8)(delayed(calculate_intersection)
                                 (fire_daily.iloc[ii:ii+1],'HRRR_GRID',0.05) 
                                 for ii in range(len(fire_daily)))
    fire_hrrr_intersection=gpd.GeoDataFrame(pd.concat(hrrr_intersections, ignore_index=True))
    fire_hrrr_intersection.set_geometry(col='geometry')
    toc = time.time()
    print(toc-tic)
    fire_hrrr_intersection = fire_hrrr_intersection.set_index([str(day_start_hour)+ 'Z Start Day', 'row', 'col'])
    fire_hrrr_intersection_xr = fire_hrrr_intersection.to_xarray()
    print(fire_hrrr_intersection_xr['weights'].sum(dim=str(day_start_hour)+ 'Z Start Day'))
    
    #load in all the HRRR days we want
    times = pd.date_range(np.datetime64(df[str(day_start_hour)+ 'Z Start Day'].iloc[0]),
                        np.datetime64(df[str(day_start_hour)+ 'Z Start Day'].iloc[len(df)-1])+
                        np.timedelta64(1,'D'))
    files,times_used = make_file_namelist(times,'/data2/lthapa/ML_daily/pygraf/Processed_HRRR_YYYYMMDDHH.nc')
    dat_hrrr = xr.open_mfdataset(files,concat_dim='time',combine='nested',compat='override', coords='all')
    dat_hrrr = dat_hrrr.assign_coords({'time': times})

    #select by time and location
    hrrr_daily_mean = dat_hrrr['veggie'].resample(time='24H',base=day_start_hour, label='left').mean(dim='time') #take the daily mean        
        
    hrrr_daily_mean_region = hrrr_daily_mean.sel(grid_yt = np.unique(fire_hrrr_intersection_xr['row'].values),
                                                 grid_xt = np.unique(fire_hrrr_intersection_xr['col'].values),
                                                 time = pd.to_datetime(fire_hrrr_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values+
                                                 'T'+str(day_start_hour)+':00:00')) #get the location of the overlaps
    
    
    df_igbp['day'] = pd.to_datetime(fire_hrrr_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values+
                                                 'T'+str(day_start_hour)+':00:00')
    
    weights_fire = fire_hrrr_intersection_xr['weights'].values
    veggie_fire = hrrr_daily_mean_region.values
    
    for vt in range(1,18):
        weights_by_veg_type= np.where(veggie_fire==vt,weights_fire,0)
        #print(np.nansum(weights_by_veg_type,axis=(1,2))) #this is what we want to return
        df_igbp['Fraction_'+varis[vt-1].replace(' ', '_')] = np.nansum(weights_by_veg_type,axis=(1,2))
        
    
    
    df_igbp_choose_max = df_igbp[['Fraction_Evergreen_Needleleaf_Forests','Fraction_Evergreen_Broadleaf_Forests', 'Fraction_Deciduous_Needleleaf_Forests',
                            'Fraction_Deciduous_Broadleaf_Forests', 'Fraction_Mixed_Forests','Fraction_Closed_Shrublands',
                            'Fraction_Open_Shrublands','Fraction_Woody_Savannas','Fraction_Savannas',
                            'Fraction_Grasslands','Fraction_Permanent_Wetlands','Fraction_Croplands',
                            'Fraction_Urban_and_Builtup_Lands','Fraction_Cropland_Natural_Vegetation_Mosaics','Fraction_Permanent_Snow_and_Ice',
                            'Fraction_Barren','Fraction_Water_Bodies']]
    
    return df_igbp

   

### ESI

In [ ]:
def esi_timeseries(df, day_start_hour):
    #preallocate space for the output
    df_esi = pd.DataFrame({'day':np.zeros(len(df)),'ESI':np.zeros(len(df))})
    
    #do the intersection, in parallel
    tic=time.time()
    esi_intersections = Parallel(n_jobs=8)(delayed(calculate_intersection)
                                 (fire_daily.iloc[ii:ii+1],'ESI_GRID',0.05) 
                                 for ii in range(len(fire_daily)))
    toc = time.time()
    print(toc-tic)
    
    fire_esi_intersection=gpd.GeoDataFrame(pd.concat(esi_intersections, ignore_index=True))
    fire_esi_intersection.set_geometry(col='geometry')
    
    
    fire_esi_intersection = fire_esi_intersection.set_index([str(day_start_hour)+ 'Z Start Day', 'lat', 'lon'])
    
    fire_esi_intersection_xr = fire_esi_intersection.to_xarray()
    print(fire_esi_intersection_xr['weights'].sum(dim=['lat','lon']))
    

    #load in esi data associated with the fire
    times = pd.date_range(np.datetime64(df[str(day_start_hour)+ 'Z Start Day'].iloc[0]),
                        np.datetime64(df[str(day_start_hour)+ 'Z Start Day'].iloc[len(df)-1])+
                        np.timedelta64(1,'D'))
    esi_filenames, esi_times = make_file_namelist(times,'/data2/lthapa/DFPPM_4WK_YYYYJJJ.nc')
    
    print(esi_filenames)
    print(esi_times)
    
    
    #open the esi files
    dat_esi = xr.open_mfdataset(esi_filenames,concat_dim='time',combine='nested',compat='override', coords='all')
    dat_esi = dat_esi.assign_coords({'time': esi_times}) #assign coords so we can resample along time
    dat_esi = dat_esi.where(dat_esi['Band1']!=-9999)
    dat_esi_daily = dat_esi.reindex(time=times,method='nearest')
    dat_esi_daily_sub = dat_esi_daily.sel(lat = fire_esi_intersection_xr['lat'].values, 
                                          lon = fire_esi_intersection_xr['lon'].values,
                      time = pd.to_datetime(fire_esi_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values), method='nearest')
                                          

    
    

    df_esi['day'].iloc[:] = pd.to_datetime(fire_esi_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values)
    varis=['ESI']
    for var in varis:
        df_esi[var] = np.nansum(fire_esi_intersection_xr['weights'].values*dat_esi_daily_sub['Band1'].values, axis=(1,2))
    
    return df_esi
    

### Resources/Personel (different format because database is excel spreadsheets, not netcdf files)

In [9]:
def resources_timeseries(df, day_start_hour,sit209_data):

    #get the fire incident number, lat, and lon
    incident_number = df['Incident Number'].iloc[0]
    fire_lat = df['Lat Fire'].iloc[0]
    fire_lon = df['Lon Fire'].iloc[0]
    #print(incident_number, fire_lat, fire_lon)
    
    sit209_data_fire = sit209_data[sit209_data['INC_IDENTIFIER']==incident_number]
    #print(sit209_data_fire)
    #do the time zone conversion
    obj=TimezoneFinder() #initialize the timezone finder
    tz = obj.timezone_at(lng=fire_lon, lat=fire_lat) #get the timezone
    local = pytz.timezone(tz)
    utc = pytz.utc
    
    #put the start and end times in local time
    loc_dt_start = [local.localize(datetime.strptime(date, '%m/%d/%Y %H:%M:%S %p')) for date in sit209_data_fire['REPORT_FROM_DATE'].values]
    loc_dt_end = [local.localize(datetime.strptime(date, '%m/%d/%Y %H:%M:%S %p')) for date in sit209_data_fire['REPORT_TO_DATE'].values]
    
    #put them in UTC time
    utc_dt_start = [time_start.astimezone(utc) for time_start in loc_dt_start]
    utc_dt_end = [time_end.astimezone(utc) for time_end in loc_dt_end]
    
    start_day = pd.to_datetime(utc_dt_start[0]).strftime('%Y-%m-%d')+' '+str(day_start_hour)+':00'
    
    
    #reassign to UTC time, this DOES keep track of daylight savings (eg +7 is used for PDT, +8 is used for PST)
    sit209_data_fire['Report Start UTC'] = pd.to_datetime(utc_dt_start)
    sit209_data_fire['Report End UTC'] = pd.to_datetime(utc_dt_end)
    sit209_data_fire['Timezone']= tz
    
    #localise the index
    sit209_data_fire = sit209_data_fire.set_index(['Report Start UTC']).tz_localize(None)
    #print(sit209_data_fire.iloc[0:4])
    
    
    ## do the 12z-12z day grouping, based on the UTC times
    #start_day_utc = str(utc_dt_start[0])
    start_day_utc=str(df[str(day_start_hour)+'Z Start Day'][0])
    start_datetime_utc = np.datetime64(start_day_utc[0:10]+'T'+str(day_start_hour).zfill(2)+':00')
    print(start_datetime_utc)
    #sit209_data_fire = sit209_data_fire.resample('24H',origin=start_datetime_utc)

    personnel = sit209_data_fire['RESOURCE_PERSONNEL'].resample('24H',origin=start_datetime_utc).sum().reset_index()
    percent_contained = sit209_data_fire['PCT_CONTAINED_COMPLETED'].resample('24H',origin=start_datetime_utc).mean().reset_index()
    df_sit209 = pd.concat([percent_contained,personnel.drop(columns='Report Start UTC')],axis=1)
    df_sit209.columns=['day', 'percent_contained', 'personnel']
    df_sit209['day'] = pd.to_datetime(df_sit209['day'].values).strftime('%Y-%m-%d')
    inds = df_sit209['day'].isin(df[str(day_start_hour)+'Z Start Day']).values
    
    return df_sit209[inds]
    

### Fuel Loading (1km

In [25]:
def fuel_loading_timeseries(df, day_start_hour):
    #do the intersection, in parallel
    tic = time.time()
    fuel_loading_intersections = Parallel(n_jobs=8)(delayed(calculate_intersection)
                                 (fire_daily.iloc[ii:ii+1],'FUEL_LOADING_GRID',0.05) 
                                 for ii in range(len(fire_daily)))
    toc = time.time()
    print(toc-tic)
    
    fire_fuel_loading_intersection=gpd.GeoDataFrame(pd.concat(fuel_loading_intersections, ignore_index=True))
    fire_fuel_loading_intersection.set_geometry(col='geometry')
    fire_fuel_loading_intersection = fire_fuel_loading_intersection.set_index([str(day_start_hour)+ 'Z Start Day', 'lat', 'lon'])
    
    fire_fuel_loading_intersection_xr = fire_fuel_loading_intersection.to_xarray()
    
    #load in fuel load data associated with the fire (it's only one dataset)
    
    #open the fuel loading file
    path_fuel_loading = '/data/lthapa/GSL_Internship/fuel_loading_FCCS16.nc'
    dat_fuel_loading = xr.open_dataset(path_fuel_loading) #map is fixed in time
    
    
    dat_fuel_loading = dat_fuel_loading.assign_coords({'time': pd.to_datetime(fire_fuel_loading_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values)})
    dat_fuel_loading = dat_fuel_loading.where(dat_fuel_loading['fuel_loadings']!=-9999)    
    dat_fuel_loading_daily = dat_fuel_loading['fuel_loadings'].expand_dims({'time': pd.to_datetime(fire_fuel_loading_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values)}) #the PWS expanded over all the days
    
    dat_fuel_loading_daily_sub = dat_fuel_loading_daily.sel(lat = fire_fuel_loading_intersection_xr['lat'].values, 
                                          lon = fire_fuel_loading_intersection_xr['lon'].values,
                      time = pd.to_datetime(fire_fuel_loading_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values), method='nearest')
                                          
    ndays = len(fire_fuel_loading_intersection_xr[str(day_start_hour)+ 'Z Start Day'])
    
    #preallocate space for the output
    df_fuel_loading = pd.DataFrame({'day':np.zeros(ndays),'fuel_loadings':np.zeros(ndays)})

    df_fuel_loading['day'].iloc[:] = pd.to_datetime(fire_fuel_loading_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values)
    varis=['fuel_loadings']
    for var in varis:
        df_fuel_loading[var] = np.nansum(fire_fuel_loading_intersection_xr['weights'].values*dat_fuel_loading_daily_sub.values, axis=(1,2))
    
    return df_fuel_loading
    

### Fire Area (ha and ac)

In [47]:
def fire_area_timeseries(df,day_start_hour):
    df_fire_area = df[[str(start_time)+'Z Start Day','fire area (ha)']].rename(columns={str(start_time)+'Z Start Day':'day'})
    df_fire_area['fire area (m2)'] = df_fire_area['fire area (ha)']*10000
    df_fire_area['fire area (acre)'] = df_fire_area['fire area (ha)']*2.471
    return df_fire_area

## Dataset-Independent Helper Functions

In [2]:
#makes and saves a geodataframe of a grid given the center and corner points for that grid as 2D matrices
def build_one_gridcell(LAT_COR, LON_COR, LAT_CTR, LON_CTR, loc):
    #print(ii,jj,count)
    ii=loc[0]
    jj=loc[1]
    #print(LAT_CTR[ii,jj], LON_CTR[ii,jj]) #ctr
    sw = (LON_COR[ii, jj],LAT_COR[ii, jj]) #SW
    se =(LON_COR[ii, jj+1],LAT_COR[ii, jj+1]) #SE
    nw = (LON_COR[ii+1, jj],LAT_COR[ii+1, jj]) #NW
    ne = (LON_COR[ii+1, jj+1],LAT_COR[ii+1, jj+1]) #NE
            
    poly_cell = Polygon([sw,nw,ne,se])
    
    return LAT_CTR[ii,jj], LON_CTR[ii,jj],ii,jj,poly_cell
    

In [3]:
#poly is the polygon for one timestep (in lcc)
#grid is an xarray of a model grid from the nc file
#grid_names is a string array [0:'lat_center_name',1:'lon_center_name',2:'lat_corner_name',3:'lon_corner_name']

def calculate_intersection(poly,dataset_name,bf):
    #load in the merra grid
    grid = xr.open_dataset(dataset_name+'.nc')

    #get the bounds of the buffered polygons
    poly_latlon =poly.to_crs(epsg=4326)
    bounds = poly_latlon.buffer(bf).bounds
    
    #first check for rows and cols
    [rows,cols] = np.where((grid.LAT_CTR.values>bounds['miny'].values)&
                    (grid.LAT_CTR.values<bounds['maxy'].values)&
                    (grid.LON_CTR.values>bounds['minx'].values)&
                    (grid.LON_CTR.values<bounds['maxx'].values))
    #print(rows,cols)
    
    locs = zip(rows,cols)
    #print([loc for loc in locs])
    
    """
    if (rows.size==0)|(cols.size==0):
        lat_middle = (bounds['maxy'].values+bounds['miny'].values)/2
        lon_middle = (bounds['maxx'].values+bounds['minx'].values)/2
        
        distance = np.sqrt((grid.LAT_CTR.values-lat_middle)**2+(grid.LON_CTR.values-lon_middle)**2)
        row_minloc,col_minloc = np.where(distance==np.min(distance))
        print(row_minloc,col_minloc)
        
        rows = np.arange(row_minloc-1,row_minloc+2,1)
        cols = np.arange(col_minloc-1,col_minloc+2,1)
        
        locs = [(row,col) for row in rows for col in cols]
        
    #print(rows,cols)
    #print(locs)
    """
    
    #make a geodataframe (in paralell of the rows and cols)
    results = Parallel(n_jobs=8)(delayed(build_one_gridcell)
                                 (grid['LAT_COR'].values, grid['LON_COR'].values,
                                  grid['LAT_CTR'].values, grid['LON_CTR'].values,loc) 
                                 for loc in locs)

    #format the grid subset into a dataframs
    df_grid=gpd.GeoDataFrame(results)
    df_grid.columns = ['lat', 'lon', 'row', 'col', 'geometry']
    df_grid.set_geometry(col='geometry',inplace=True,crs='EPSG:4326') #need to say it's in lat/lon before transform to LCC
    df_grid=df_grid.to_crs(epsg=3347)
    
    #intersect the polygon with the grid subset
    intersection = gpd.overlay(df_grid, poly, how='intersection',keep_geom_type=False).drop_duplicates()
    intersection['grid intersection area (ha)'] =intersection['geometry'].area/10000
    intersection['weights'] = intersection['grid intersection area (ha)']/intersection['fire area (ha)'] 
    
    return intersection


In [4]:
def make_file_namelist(time,base_filename):
    filename_list = np.array([])
    times_back_used = np.array([])
    for jj in range(len(time)):
        fname = base_filename.replace('YYYY',time[jj].strftime('%Y')).\
                                replace('MM',time[jj].strftime('%m')).\
                                replace('DD',time[jj].strftime('%d')).\
                                replace('HH',time[jj].strftime('%H')).\
                                replace('JJJ',time[jj].strftime('%j'))
        if exists(fname):
            filename_list = np.append(filename_list,fname)
            times_back_used = np.append(times_back_used,time[jj])
    return filename_list, times_back_used

## Check that the buffers work

In [ ]:
#all fires
#fire_incidents = ['BOBCAT', 'DOLAN', 'HOLIDAY FARM','CREEK', 'LAKE', 'CAMERON PEAK', 'PINE GULCH', 'WILLIAMS FLATS', 'SHADY','PEDRO MOUNTAIN', 'WALKER', '204 COW']

#2020 fires
#fire_incidents = ['AUGUST COMPLEX','BOBCAT', 'DOLAN', 'HOLIDAY FARM','CREEK', 'LAKE', 'CAMERON PEAK', 'PINE GULCH']
#fire_incidents=['BOBCAT', 'DOLAN', 'HOLIDAY FARM','CREEK', 'LAKE', 'CAMERON PEAK', 'PINE GULCH']
fire_incidents = ['LAKE']

path_poly = '/data2/lthapa/ML_daily/fire_polygons/'
suffix_poly = 'Z_day_start.geojson'
start_time=12
for jj in range(len(fire_incidents)):
    fire_name = fire_incidents[jj].lower().replace(' ','_')
    print(path_poly+fire_name+'_VIIRS_daily_'+str(start_time)+suffix_poly)
    fire_daily = gpd.read_file(path_poly+fire_name+'_VIIRS_daily_'+str(start_time)+suffix_poly)
    fire_daily=fire_daily.drop(columns=['Current Overpass'])
    fire_daily = fire_daily.drop(np.where(fire_daily['geometry']==None)[0])
    fire_daily['fire area (ha)'] = fire_daily['geometry'].area/10000 #hectares
    fire_daily.set_geometry(col='geometry', inplace=True) #designate the geometry column
    fire_daily = fire_daily.rename(columns={'Current Day':'UTC Day', 'Local Day': str(start_time)+ 'Z Start Day'})
    
    fire_daily = fire_daily.iloc[np.array(fire_daily['UTC Day'].values,dtype='datetime64')<=np.datetime64('2020-10-31'),:]

    #print(fire_daily)
    
    #merra
    tic = time.time()
    merra_intersections = Parallel(n_jobs=8)(delayed(calculate_intersection)
                                 (fire_daily.iloc[ii:ii+1],'ESI_GRID',0.05) 
                                 for ii in range(len(fire_daily)))
    toc = time.time()
    print(toc-tic)
    fire_merra_intersection=gpd.GeoDataFrame(pd.concat(merra_intersections, ignore_index=True))
    fire_merra_intersection.set_geometry(col='geometry')
    fire_merra_intersection = fire_merra_intersection.set_index([str(start_time)+ 'Z Start Day', 'lat', 'lon'])
    print(fire_merra_intersection)
    fire_merra_intersection_xr = fire_merra_intersection.to_xarray()
    print(fire_merra_intersection_xr['weights'].sum(dim=['lat','lon']).values)
    

    esi_intersections = Parallel(n_jobs=8)(delayed(calculate_intersection)
                                 (fire_daily.iloc[ii:ii+1],'ESI_GRID',0.05) 
                                 for ii in range(len(fire_daily)))
    toc = time.time()
    fire_esi_intersection=gpd.GeoDataFrame(pd.concat(esi_intersections, ignore_index=True))
    fire_esi_intersection.set_geometry(col='geometry')
    fire_esi_intersection = fire_esi_intersection.set_index([str(start_time)+ 'Z Start Day', 'lat', 'lon'])
    fire_esi_intersection_xr = fire_esi_intersection.to_xarray()
    print(fire_esi_intersection_xr['weights'].sum(dim=['lat','lon']))

# Testing

In [ ]:
start_time=12
fire_daily = gpd.read_file('./fire_polygons/cameron_peak_VIIRS_daily_12Z_day_start.geojson')
#fire_daily = gpd.read_file('./fire_polygons/august_complex_VIIRS_daily_12Z_day_start.geojson')
#fire_daily = gpd.read_file('./fire_polygons/williams_flats_VIIRS_daily_12Z_day_start.geojson')
fire_daily=fire_daily.drop(columns=['Current Overpass'])
fire_daily = fire_daily.drop(np.where(fire_daily['geometry']==None)[0])
fire_daily['fire area (ha)'] = fire_daily['geometry'].area/10000 #hectares
fire_daily.set_geometry(col='geometry', inplace=True) #designate the geometry column
fire_daily = fire_daily.rename(columns={'Current Day':'UTC Day', 'Local Day': str(start_time)+ 'Z Start Day'})
fire_daily = fire_daily.iloc[np.array(fire_daily['UTC Day'].values,dtype='datetime64')<=np.datetime64('2020-10-31'),:]
print(fire_daily)

In [ ]:
#hrrr_timeseries(fire_daily.iloc[15:25],12)

times_intersect = np.unique(fire_daily[str(12)+ 'Z Start Day'].iloc[39:40].values)
for today in times_intersect:
    print(today)
    times_back = pd.date_range(start=np.datetime64(today)-np.timedelta64(5,'D'), end=np.datetime64(today)+
                                   np.timedelta64(1,'D'),freq='H')
    files_back,times_back_used = make_hrrr_file_namelist(times_back)
    
    #print(files_back)
    dat_hrrr = xr.open_mfdataset(files_back,concat_dim='time',combine='nested',compat='override', coords='all')
    dat_hrrr = dat_hrrr.assign_coords({'time': times_back_used})
    hrrr_daily_mean = dat_hrrr.resample(time='24H',base=12, label='left').mean(dim='time') #take the daily mean        

    print(hrrr_daily_mean.time)
    dat_hrrr.close()

In [ ]:
ints = calculate_intersection(fire_daily.iloc[11:12],'ESI_GRID',0.05)
print(ints.sum())
#tic = time.time()
#hrrr_intersections = Parallel(n_jobs=8)(delayed(calculate_intersection)
#                                 (fire_daily.iloc[ii:ii+1],'HRRR_GRID',0.01) 
#                                 for ii in range(len(fire_daily)))
#toc=time.time()
#print(toc-tic)

In [ ]:
grid = xr.open_dataset('PWS_GRID.nc')
print(grid.LAT_CTR[0:5,0:5])

In [ ]:
grid = xr.open_dataset('HRRR_GRID.nc')
bf = 0.05

for ii in range(len(fire_daily)):
    poly=fire_daily[ii:ii+1]
    #get the bounds of the buffered polygons
    poly_latlon =poly.to_crs(epsg=4326)
    bounds = poly_latlon.buffer(bf).bounds
    
    #first check for rows and cols
    [rows,cols] = np.where((grid.LAT_CTR.values>bounds['miny'].values)&
                    (grid.LAT_CTR.values<bounds['maxy'].values)&
                    (grid.LON_CTR.values>bounds['minx'].values)&
                    (grid.LON_CTR.values<bounds['maxx'].values))
    print(rows,cols)